In [25]:
import cv2 as cv
import numpy as np
from skimage.feature import hog
import pandas as pd
import numpy as np
#"/Users/Mehr/Documents/projects/tree/1.jpg"
import cv2 as cv


In [57]:
from skimage import io, transform, img_as_ubyte
import os
import numpy as np

def resize_image_skimage(input_dir, output_dir, target_size):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for filename in os.listdir(input_dir):
        if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
            image_path = os.path.join(input_dir, filename)
            try:
                image = io.imread(image_path)
                resized_image = transform.resize(image, target_size, anti_aliasing=True)
                
                # Convert RGBA to RGB
                if resized_image.shape[-1] == 4:
                    resized_image = resized_image[..., :3]
                
                resized_image = img_as_ubyte(resized_image)
                output_path = os.path.join(output_dir, os.path.splitext(filename)[0] + ".jpg")
                io.imsave(output_path, resized_image)
            except Exception as e:
                print(f"Error processing {image_path}: {e}")

input_directory = '/Users/Mehr/Documents/projects/tree/'
output_directory = '/Users/Mehr/Documents/projects/tree/resized/'
target_size = (224, 224)  # Example target size

#resize_image_skimage(input_directory, output_directory, target_size)


In [49]:
import cv2 as cv
from skimage.feature import hog
import os

def extract(path):
    image_path = os.path.join("/Users/Mehr/Documents/projects/tree/resized/", path)
    image = cv.imread(image_path)

    # Check if the image was loaded successfully
    if image is None:
        print(f"Error: Could not load image at {image_path}")
        return None

    # Convert to grayscale
    image = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
    features, hog_image = hog(image, pixels_per_cell=(16, 16), cells_per_block=(2, 2), visualize=True, feature_vector=True)
    return features

# Example usage
path = "1.jpg"  # Replace with your actual image filename
features = extract(path)
if features is not None:
    print("Features extracted successfully")
else:
    print("Failed to extract features")


Features extracted successfully


In [52]:
import cv2
import numpy as np
import pandas as pd
from skimage.feature import hog
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error,r2_score

# Define the feature extraction function
def extract_features(image_path):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    features, hog_image = hog(image, pixels_per_cell=(16, 16), cells_per_block=(2, 2), visualize=True, feature_vector=True)
    return features

# Example data
# data = pd.DataFrame({
#     'filenames': ['image1.jpg', 'image2.jpg'],  # Replace with your actual filenames
#     'tree_count': [5, 3]  # Replace with your actual tree counts
# })

data=pd.read_csv("/Users/Mehr/Documents/projects/tree/resized/tree.csv")
data=data[:4]

def extract(path):
    image=cv.imread("/Users/Mehr/Documents/projects/tree/resized/"+path)
    image=cv.cvtColor(image,cv.COLOR_BGR2GRAY)
    features, hog_image = hog(image, pixels_per_cell=(16, 16), cells_per_block=(2, 2), visualize=True,feature_vector=True)
    return features

# Extract features and check lengths
data['features'] = data['filenames'].apply(extract)
feature_lengths = data['features'].apply(len)

# Ensure all feature vectors have the same length
if len(feature_lengths.unique()) == 1:
    X = np.array(data['features'].tolist())
    y = data['tree_count'].values
    
    # Split the data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Train the model
    model = RandomForestRegressor(n_estimators=100)
    model.fit(X_train, y_train)
    
    # Make predictions
    predictions = model.predict(X_test)
    mse = mean_squared_error(y_test, predictions)
    print(f"Mean Squared Error: {mse}")
else:
    print("Inconsistent feature lengths detected:", feature_lengths.unique())


Mean Squared Error: 0.9025000000000054


In [55]:
def predict_tree_count(image_path,model):
    features=extract(image_path)
    features=np.array(features).reshape(1,-1)
    prediction=model.predict(features)
    return prediction[0]

new_image_path = "/Users/Mehr/Documents/projects/tree/resized/test.jpg"
    
    # Predict the tree count in the new image
predicted_count = predict_tree_count(new_image_path, model)
print(f"Predicted number of trees in the new image: {predicted_count}")


[ WARN:0@43152.063] global loadsave.cpp:241 findDecoder imread_('/Users/Mehr/Documents/projects/tree/resized//Users/Mehr/Documents/projects/tree/resized/test.jpg'): can't open/read file: check file path/integrity


error: OpenCV(4.10.0) /private/var/folders/vc/3kfwh15d7234657gp2jqxqj80000gn/T/pip-install-48fn8dze/opencv-python_137120336ea840aa8b8cceab99cf30f3/opencv/modules/imgproc/src/color.cpp:196: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'
